In [1]:
from dask_jobqueue import PBSCluster

In [2]:
cluster = PBSCluster(cores=8, memory='50GB', walltime='02:00:00', queue='economy', project='NIOW0001')

In [3]:
!qstat -u $USER


chadmin1: 
                                                            Req'd  Req'd   Elap
Job ID          Username Queue    Jobname    SessID NDS TSK Memory Time  S Time
--------------- -------- -------- ---------- ------ --- --- ------ ----- - -----
2939754.chadmin abanihi  economy  STDIN       50301   1   2    --  03:00 R 01:38


In [4]:
cluster.scale(5)

In [5]:
!qstat -u $USER


chadmin1: 
                                                            Req'd  Req'd   Elap
Job ID          Username Queue    Jobname    SessID NDS TSK Memory Time  S Time
--------------- -------- -------- ---------- ------ --- --- ------ ----- - -----
2939754.chadmin abanihi  economy  STDIN       50301   1   2    --  03:00 R 01:38
2941107.chadmin abanihi  economy  dask-worke  68964   1   1    --  02:00 R 00:00
2941108.chadmin abanihi  economy  dask-worke   2985   1   1    --  02:00 R 00:00
2941109.chadmin abanihi  economy  dask-worke  63484   1   1    --  02:00 R 00:00
2941110.chadmin abanihi  economy  dask-worke  70812   1   1    --  02:00 R 00:00
2941111.chadmin abanihi  economy  dask-worke  59581   1   1    --  02:00 R 00:00


In [6]:
from dask.distributed import Client
client = Client(cluster)
client

In [7]:
client = Client(cluster)
client

Client Scheduler: tcp://10.148.1.14:48637 Dashboard: http://10.148.1.14:8787/status,Cluster Workers: 5 Cores: 40 Memory: 250.00 GB


In [11]:
input_test = "/glade/scratch/abanihi/lens-dataset-test/input/T/*.nc"

In [12]:
import xarray as xr 
import zarr 

In [15]:
xr.open_dataset('/glade/scratch/abanihi/lens-dataset-test/input/T/b.e11.BRCP85C5CNBDRD.f09_g16.034_CMIP5-O3.cam.h0.T.200601-210012.nc', 
               engine='netcdf4').info()

xarray.Dataset {
dimensions:
	cosp_ht = 40 ;
	cosp_htmisr = 16 ;
	cosp_prs = 7 ;
	cosp_scol = 10 ;
	cosp_sr = 15 ;
	cosp_sza = 5 ;
	cosp_tau = 7 ;
	cosp_tau_modis = 6 ;
	ilev = 31 ;
	lat = 192 ;
	lev = 30 ;
	lon = 288 ;
	nbnd = 2 ;
	slat = 191 ;
	slon = 288 ;
	time = 1140 ;

variables:
	float64 lev(lev) ;
		lev:long_name = hybrid level at midpoints (1000*(A+B)) ;
		lev:units = level ;
		lev:positive = down ;
		lev:standard_name = atmosphere_hybrid_sigma_pressure_coordinate ;
		lev:formula_terms = a: hyam b: hybm p0: P0 ps: PS ;
	float64 hyam(lev) ;
		hyam:long_name = hybrid A coefficient at layer midpoints ;
	float64 hybm(lev) ;
		hybm:long_name = hybrid B coefficient at layer midpoints ;
	float64 ilev(ilev) ;
		ilev:long_name = hybrid level at interfaces (1000*(A+B)) ;
		ilev:units = level ;
		ilev:positive = down ;
		ilev:standard_name = atmosphere_hybrid_sigma_pressure_coordinate ;
		ilev:formula_terms = a: hyai b: hybi p0: P0 ps: PS ;
	float64 hyai(ilev) ;
		hyai:long_name = hybrid

In [16]:
dset = xr.open_mfdataset(input_test, engine='netcdf4', concat_dim='time')

In [17]:
dset.info()

xarray.Dataset {
dimensions:
	cosp_ht = 40 ;
	cosp_htmisr = 16 ;
	cosp_prs = 7 ;
	cosp_scol = 10 ;
	cosp_sr = 15 ;
	cosp_sza = 5 ;
	cosp_tau = 7 ;
	cosp_tau_modis = 6 ;
	ilev = 31 ;
	lat = 192 ;
	lev = 30 ;
	lon = 288 ;
	nbnd = 2 ;
	slat = 191 ;
	slon = 288 ;
	time = 2280 ;

variables:
	float64 lev(lev) ;
		lev:long_name = hybrid level at midpoints (1000*(A+B)) ;
		lev:units = level ;
		lev:positive = down ;
		lev:standard_name = atmosphere_hybrid_sigma_pressure_coordinate ;
		lev:formula_terms = a: hyam b: hybm p0: P0 ps: PS ;
	float64 ilev(ilev) ;
		ilev:long_name = hybrid level at interfaces (1000*(A+B)) ;
		ilev:units = level ;
		ilev:positive = down ;
		ilev:standard_name = atmosphere_hybrid_sigma_pressure_coordinate ;
		ilev:formula_terms = a: hyai b: hybi p0: P0 ps: PS ;
	float64 cosp_prs(cosp_prs) ;
		cosp_prs:long_name = COSP Mean ISCCP pressure ;
		cosp_prs:units = mb ;
		cosp_prs:bounds = cosp_prs_bnds ;
	float64 cosp_tau(cosp_tau) ;
		cosp_tau:long_name = COSP Mean ISCCP op

In [18]:
cluster.close()

In [19]:
client.close()

tornado.application - ERROR - Exception in Future <Future cancelled> after timeout
Traceback (most recent call last):
  File "/glade/u/home/abanihi/.conda/envs/pangeo/lib/python3.6/site-packages/tornado/gen.py", line 970, in error_callback
    future.result()
concurrent.futures._base.CancelledError
